In [ ]:
# 動かなかった　未完成
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

csv = pd.read_csv("CSV/ref.csv")
df = pd.DataFrame(csv["USD"])
df

k = 0
for p in range(15): 
    for i in range(25):
        X_train,y_train =df[k + i]
# モデルの作成
    model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
    ])

    model.compile(loss="mse", optimizer="adam")
    model.summary()
    history = model.fit(X_train, y_train, epochs=20,
    validation_data=(X_valid, y_valid))
    model.evaluate(X_valid, y_valid)


    # グラフの表示
    def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$", legend=True):
        plt.plot(series, ".-")
        if y is not None:
            plt.plot(n_steps, y, "bo", label="Target")
        if y_pred is not None:
            plt.plot(n_steps, y_pred, "rx", markersize=10, label="Prediction")
            plt.grid(True)
        if x_label:
            plt.xlabel(x_label, fontsize=16)
        if y_label:
            plt.ylabel(y_label, fontsize=16, rotation=0)
            plt.hlines(0, 0, 100, linewidth=1)
            plt.axis([0, n_steps + 1, -1, 1])
        if legend and (y or y_pred):
            plt.legend(fontsize=14, loc="upper left")
    y_pred = model.predict(X_valid)
    plot_series(X_valid[0, :, 0], y_valid[0, 0], y_pred[0, 0])
    plt.show()
    k = k +25


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sktime.forecasting.model_selection import temporal_train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score

# データロード
df = pd.read_csv("CSV/ref.csv")
x = df["USD"].values

# スケーリング変換
x = x.reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

# 目的変数と説明変数の作成
def make_dataset( indata, timestep ):
    data, target = [], []
    for i in range(len(indata)-timestep):
        data.append(indata[i:i + timestep])
        target.append(indata[i + timestep])
        re_data = np.array(data).reshape(len(data), timestep, 1)
        re_target = np.array(target).reshape(len(data), 1)
        return re_data, re_target

X, y = make_dataset(x_scaled, 25)

# 訓練セットと評価セットの作成
X_train, X_test, y_train, y_test = temporal_train_test_split( X, y, test_size=1000 )
X_train, X_valid, y_train, y_valid = temporal_train_test_split( X, y, test_size=1000 )
# X_train ,X_test, X_valid = temporal_train_test_split(X,test_size=0.1)
# y_train,y_test,y_valid = temporal_train_test_split(y,test_size= 0.1)

# 学習モデルの生成
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential([
keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
keras.layers.SimpleRNN(20),
keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer="adam")
model.summary()
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

# 評価
model.evaluate(X_valid, y_valid)
pred = model.predict(X_test)
pred_train = model.predict(X_train)
print("Test data R2 score: %.4f" % (r2_score(y_valid.flatten(), pred.flatten())))
print("Train data R2 score: %.4f" % (r2_score(y_train.flatten(), pred_train.flatten())))

# 学習曲線
plt.plot(history.history['loss'],label="train set")
plt.plot(history.history['val_loss'],label="test set")
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

# 予測結果
plt.plot(y_valid,label="y")
plt.plot(pred,label="pred")
plt.legend()
plt.show()

ValueError: test_size=1000 should be either positive and smaller than the number of samples 1 or a float in the (0, 1) range